In [1]:
import pickle
import os
import numpy as np
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

CIFAR_DIR = './cifar-10-batches-py'
print(os.listdir(CIFAR_DIR))

def loadData(filename):
    with open(filename, "rb") as f:
        data = pickle.load(f, encoding='latin1')
        return data['data'], data['labels']

2023-06-17 01:36:34.989697: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Instructions for updating:
non-resource variables are not supported in the long term
['data_batch_1', 'readme.html', 'batches.meta', 'data_batch_2', 'data_batch_5', 'test_batch', 'data_batch_4', 'data_batch_3']


In [2]:
class CifarData:
    def __init__(self, filenames, isShuffle):
        allData = []
        allLabels = []
        for filename in filenames:
            data, labels = loadData(filename)
            for d, label in zip(data, labels):
                if label in [0, 1]:
                    allData.append(d)
                    allLabels.append(label)
        self._data = np.vstack(allData)
        self._data = self._data / 127.5 - 1
        self._labels = np.hstack(allLabels)
        self._size = self._data.shape[0]
        self._startPos = 0
        self._isShuffle = isShuffle
        if isShuffle:
            self.shuffle()
            
    def shuffle(self):
        p = np.random.permutation(self._size)
        self._data = self._data[p]
        self._labels = self._labels[p]

    def nextBatch(self, size):
        endPos = self._startPos + size
        if endPos > self._size:
            if self._isShuffle:
                self.shuffle()
                self._startPos = 0
                endPos = size
            else:
                raise Exception("No more samples")
            
        if endPos > self._size:
            raise Exception("Sample size is less than %d", size)
        
        batch_data = self._data[self._startPos: endPos]
        batch_labels = self._labels[self._startPos: endPos]
        self._startPos = endPos
        return batch_data, batch_labels

train_filenames = [os.path.join(CIFAR_DIR, "data_batch_%d" % i) for i in range(1, 6)]
train_data = CifarData(train_filenames, True)
# print(training_data._size)
# print(training_data.nextBatch(10000))

test_filenames = [os.path.join(CIFAR_DIR, "test_batch")]
test_data = CifarData(test_filenames, False)
# print(test_data._size)
# print(test_data.nextBatch(2000))

In [3]:
x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])

w = tf.get_variable("w", [x.get_shape()[-1], 1], initializer = tf.random_normal_initializer(0, 1))
b = tf.get_variable("b", [1], initializer = tf.constant_initializer(0.0))

y_ = tf.matmul(x, w) + b
y_activate = tf.nn.sigmoid(y_)

y_reshape = tf.reshape(y, (-1, 1))
y_float = tf.cast(y_reshape, tf.float32)
loss = tf.reduce_mean(tf.square(y_float - y_activate))

predict = y_activate > 0.5
correct_prediction = tf.equal(tf.cast(predict, tf.int64), y_reshape)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with (tf.name_scope("train_op")):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)


In [4]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 100000
test_steps = 100

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.nextBatch(batch_size)
        loss_val, acc_val, _ = sess.run(
            [loss, accuracy, train_op],
            feed_dict = {
                x: batch_data,
                y: batch_labels
            }
        )
        
        if (i + 1) % 500 == 0:
            print("[Train] Step: %d, loss: %4.5f, acc: %4.5f" % (i + 1, loss_val, acc_val))
            
        if (i + 1) % 5000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.nextBatch(batch_size)
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data,
                        y: test_batch_labels
                    }
                )
                all_test_acc_val.append(test_acc_val)
            test_acc_val = np.mean(all_test_acc_val)
            print("[Test ] Step: %d, acc: %4.5f" % (i + 1, test_acc_val))

[Train] Step: 500, loss: 0.46847, acc: 0.50000
[Train] Step: 1000, loss: 0.17090, acc: 0.80000
[Train] Step: 1500, loss: 0.15127, acc: 0.85000
[Train] Step: 2000, loss: 0.27976, acc: 0.70000
[Train] Step: 2500, loss: 0.13905, acc: 0.85000
[Train] Step: 3000, loss: 0.39871, acc: 0.60000
[Train] Step: 3500, loss: 0.29997, acc: 0.70000
[Train] Step: 4000, loss: 0.29130, acc: 0.70000
[Train] Step: 4500, loss: 0.35709, acc: 0.65000
[Train] Step: 5000, loss: 0.19983, acc: 0.80000
[Test ] Step: 5000, acc: 0.79250
[Train] Step: 5500, loss: 0.20193, acc: 0.80000
[Train] Step: 6000, loss: 0.04741, acc: 0.95000
[Train] Step: 6500, loss: 0.23404, acc: 0.75000
[Train] Step: 7000, loss: 0.00108, acc: 1.00000
[Train] Step: 7500, loss: 0.31330, acc: 0.65000
[Train] Step: 8000, loss: 0.35001, acc: 0.65000
[Train] Step: 8500, loss: 0.20883, acc: 0.80000
[Train] Step: 9000, loss: 0.05005, acc: 0.95000
[Train] Step: 9500, loss: 0.09944, acc: 0.90000
[Train] Step: 10000, loss: 0.18929, acc: 0.80000
[Test ]

[Train] Step: 79500, loss: 0.05000, acc: 0.95000
[Train] Step: 80000, loss: 0.20001, acc: 0.80000
[Test ] Step: 80000, acc: 0.82250
[Train] Step: 80500, loss: 0.15000, acc: 0.85000
[Train] Step: 81000, loss: 0.05000, acc: 0.95000
[Train] Step: 81500, loss: 0.24793, acc: 0.75000
[Train] Step: 82000, loss: 0.15031, acc: 0.85000
[Train] Step: 82500, loss: 0.30072, acc: 0.70000
[Train] Step: 83000, loss: 0.15077, acc: 0.85000
[Train] Step: 83500, loss: 0.14990, acc: 0.85000
[Train] Step: 84000, loss: 0.25011, acc: 0.75000
[Train] Step: 84500, loss: 0.17306, acc: 0.80000
[Train] Step: 85000, loss: 0.16360, acc: 0.80000
[Test ] Step: 85000, acc: 0.82300
[Train] Step: 85500, loss: 0.20000, acc: 0.80000
[Train] Step: 86000, loss: 0.19955, acc: 0.80000
[Train] Step: 86500, loss: 0.10009, acc: 0.90000
[Train] Step: 87000, loss: 0.15127, acc: 0.85000
[Train] Step: 87500, loss: 0.15389, acc: 0.85000
[Train] Step: 88000, loss: 0.10011, acc: 0.90000
[Train] Step: 88500, loss: 0.09986, acc: 0.90000
[